In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [4]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [5]:
data.to_sql("boston", con, index=False, if_exists="replace")

506

In [6]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
pd.read_sql(
    """
    SELECT 
        sum(case when CRIM = 0 then 1 end) as CRIM,
        sum(case when ZN = 0 then 1 end) as ZN,
        sum(case when INDUS = 0 then 1 end) as INDUS,
        sum(case when CHAS = 0 then 1 end) as CHAS,
        sum(case when NOX = 0 then 1 end) as NOX
    FROM 
        boston
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,None,372,None,471,None


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [9]:
pd.read_sql(
    """
    SELECT 
        count(distinct CRIM) as CRIM,
        count(distinct ZN) as ZN,
        count(distinct INDUS) as INDUS,
        count(distinct CHAS) as CHAS,
        count(distinct NOX) as NOX
    FROM 
        boston
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по данным в этих колонках

In [10]:
pd.read_sql(
    """
    select case when min(CRIM) in ( select CRIM from boston
                       order by 1
                       limit 1
                       offset (select count(*) from boston)/ 2 ) 
                then 'CRIM'
                end 'COL'
    from boston 
    union
    select case when min(ZN) in ( select ZN from boston
                       order by 1
                       limit 1
                       offset (select count(*) from boston)/ 2 ) 
                then 'ZN'
                end 'COL'
    from boston 
    union                
    select case when min(INDUS) in ( select INDUS from boston
                       order by 1
                       limit 1
                       offset (select count(*) from boston)/ 2 ) 
                then 'INDUS'
                end 'COL'
    from boston 
    union               
    select case when min(CHAS) in ( select CHAS from boston
                       order by 1
                       limit 1
                       offset (select count(*) from boston)/ 2 ) 
                then 'CHAS'
                end 'COL'
    from boston 
    union
    select case when min(NOX) in ( select NOX from boston
                       order by 1
                       limit 1
                       offset (select count(*) from boston)/ 2 ) 
                then 'NOX'
                end 'COL'
    from boston 
    """,
    con,
)


,COL
0,None
1,CHAS
2,ZN


In [11]:
pd.read_sql(
    """
    select * from(
    select case when min(CRIM) in ( select CRIM from boston
                       order by 1
                       limit 1
                       offset (select count(*) from boston)/ 2 ) 
           then min(CRIM)
           else 'N'
           end CRIM,
           case when min(ZN) in ( select ZN from boston
                       order by 1
                       limit 1
                       offset (select count(*) from boston)/ 2 ) 
           then min(ZN)
           else 'N'
           end ZN,
           case when min(INDUS) in ( select INDUS from boston
                       order by 1
                       limit 1
                       offset (select count(*) from boston)/ 2 ) 
           then min(INDUS)
           else 'N'
           end INDUS,
           case when min(CHAS) in ( select CHAS from boston
                       order by 1
                       limit 1
                       offset (select count(*) from boston)/ 2 ) 
           then min(CHAS)
           else 'N'
           end CHAS,
           case when min(NOX) in ( select NOX from boston
                       order by 1
                       limit 1
                       offset (select count(*) from boston)/ 2 ) 
           then min(NOX)
           else 'N'
           end NOX
    from boston ) a
    where ((CRIM !='N') or (ZN !='N') or (INDUS !='N') or (CHAS !='N') or (NOX !='N'))
    """,
    con,
)


,CRIM,ZN,INDUS,CHAS,NOX
0,N,0.0,N,0.0,N


In [12]:
# Вывод: в этих 2х колонках значительно преобладает минимальное значение, оно равно 0

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница). Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [13]:
pd.read_sql(
    """
    select (
    SELECT avg(RM) FROM boston
    where MEDV = (SELECT max(MEDV)from boston)) -
       ( SELECT 
        avg(RM) 
    FROM (
        SELECT
            *
        from 
            boston
        order by MEDV  
        limit 25
    ) agg_lim
    ) as diff
    
    union all
    
    select ( SELECT 
        avg(RM) 
    FROM (
        SELECT
            *
        from 
            boston
        order by MEDV desc 
        limit 50
    ) agg_lim )
     -
       ( SELECT 
        avg(RM) 
    FROM (
        SELECT
            *
        from 
            boston
        order by MEDV  
        limit 50
    ) agg_lim
    ) as diff
    
        union all
    
    select ( SELECT 
        avg(RM) 
    FROM (
        SELECT
            *
        from 
            boston
        order by MEDV desc 
        limit 100
    ) agg_lim )
     -
       ( SELECT 
        avg(RM) 
    FROM (
        SELECT
            *
        from 
            boston
        order by MEDV  
        limit 100
    ) agg_lim
    ) as diff
    
            union all
    
    select ( SELECT 
        avg(RM) 
    FROM (
        SELECT
            *
        from 
            boston
        order by MEDV desc 
        limit 200
    ) agg_lim )
     -
       ( SELECT 
        avg(RM) 
    FROM (
        SELECT
            *
        from 
            boston
        order by MEDV  
        limit 200
    ) agg_lim
    ) as diff
    
            union all
    
    select ( SELECT 
        avg(RM) 
    FROM (
        SELECT
            *
        from 
            boston
        order by MEDV desc 
        limit 300
    ) agg_lim )
     -
       ( SELECT 
        avg(RM) 
    FROM (
        SELECT
            *
        from 
            boston
        order by MEDV  
        limit 300
    ) agg_lim
    ) as diff
    """,
    con,
)

,diff
0,1.736160
1,1.733820
2,1.311640
3,0.846975
4,0.566780


In [14]:
# Разница в кол-ве комнат падает при увеличении выборки/
# между дорогими и дешевыми домами есть разница в кол-ве комнат - в дорогих больше комнат

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [15]:
pd.read_sql(
    """
    SELECT  LSTAT, 
            MEDV,
            rank() over(partition by MEDV order by LSTAT) as rn
    FROM boston
    where MEDV = (SELECT max(MEDV)from boston)
    """,
    con,
)

,LSTAT,MEDV,rn
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,3
3,2.96,50.0,4
4,2.97,50.0,5
5,3.16,50.0,6
6,3.26,50.0,7
7,3.32,50.0,8
8,3.70,50.0,9
9,3.73,50.0,10


In [16]:
# Вывод: Дорогие дома есть в различных районах с разными статусами населения

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [17]:
pd.read_sql(
    """
    SELECT  CHAS, 
            avg(MEDV)  as avg_cost
    FROM boston
    group by 1
    """,
    con,
)

,CHAS,avg_cost
0,0.0,22.093843
1,1.0,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [18]:
pd.read_sql(
    """
    SELECT  case when avg(CRIM)> (select avg(CRIM) from boston where chas = 1) then 'CRIM' 
                 when avg(ZN)> (select avg(ZN) from boston where chas = 1) then 'ZN' 
                 when avg(INDUS)> (select avg(INDUS) from boston where chas = 1) then 'INDUS' 
                 when avg(CHAS)> (select avg(CHAS) from boston where chas = 1) then 'CHAS' 
                 when avg(NOX)> (select avg(NOX) from boston where chas = 1) then 'NOX'
            end column
    FROM boston
    """,
    con,
)

,column
0,CRIM


In [19]:
# Вывод: уровень криминала у воды ниже чем 

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [20]:
pd.read_sql(
    """
    WITH p AS (SELECT INDUS, NOX, NTILE(10) OVER (ORDER BY INDUS, NOX) AS percentile
           FROM boston)
                SELECT percentile, MAX(INDUS) as INDUS, MAX(NOX) as NOX
                FROM p
                GROUP BY percentile;
    """,
    con,
)

,percentile,INDUS,NOX
0,1,2.89,0.538
1,2,4.39,0.647
2,3,5.96,0.515
3,4,7.38,0.507
4,5,9.69,0.585
5,6,13.89,0.585
6,7,18.10,0.631
7,8,18.10,0.713
8,9,19.58,0.770
9,10,27.74,0.871


In [ ]:
#  С увеличением доли пром застройки  увеличивается концентрация оксидов азота

